## Tagging
Each dataset has different types of tagging.  
This is applied to NER tagging.  
Each dataset has its own tagging and each model uses different tokenizer.  
To calculate accuracies, we need to arrange tags to compare properly.
In this notebook, two different tokens of model and tagging are adjusted

In [1]:
# EXO-brain form
"""
## 1
## 오에 겐자부로는 일본 현대문학의 초석을 
## <오에 겐자부로:PER>는 <일본:LOC> 현대문학의 초석을 
오에	오에	NNG	B-PER
_	_	_	I-PER
겐자부로	겐자부로	NNP	I-PER
는	는	JX	O

## 2
## 이미 수상자(2000년 김대중 전 대통령)를 배출한 데다
## 이미 수상자(<2000년:DAT> <김대중:PER> 전 대통령)를 배출한 데다
이미	이미	MAG	O
_	_	_	O
수상자	수상자	NNG	O
(	(	SS	O
"""

In [149]:
test = """오에	오에	NNG	B-PER
_	_	_	I-PER
겐자부로	겐자부로	NNP	I-PER
는	는	JX	O
_	_	_	O
일본	일본	NNP	B-LOC
_	_	_	O
현대	현대	NNG	O
문학	문학	NNG	O
의	의	JKG	O
_	_	_	O
초석	초석	NNG	O
을	을	JKO	O
_	_	_	O
놓	놓	VV	O
은	은	ETM	O
_	_	_	O
것	것	NNB	O
으로	으로	JKB	O
_	_	_	O
평가	평가	NNG	O
받	받	VV	O
는	는	ETM	O
_	_	_	O
작가	작가	NNG	O
_	_	_	O
나쓰메	나쓰메	NNP	B-PER
_	_	_	I-PER
소세키	소세키	NNP	I-PER
(	(	SS	O
1867	1867	SN	B-DUR
~	~	SO	I-DUR
1916	1916	SN	I-DUR
)	)	SS	O
의	의	JKG	O
_	_	_	O
대표작	대표작	NNG	O
_	_	_	O
‘	‘	SS	O
마음	마음	NNG	B-POH
’	’	SS	O
에	에	JKB	O
_	_	_	O
담긴	담기+ㄴ	VV+ETM	O
_	_	_	O
군국주의	군국주의	NNG	O
적	적	XSN	O
_	_	_	O
요소	요소	NNG	O
,	,	SP	O
_	_	_	O
야스쿠니	야스쿠니	NNP	B-ORG
_	_	_	I-ORG
신사	신사	NNG	I-ORG
_	_	_	O
참배	참배	NNG	O
_	_	_	O
행위	행위	NNG	O
까지	까지	JX	O
_	_	_	O
소설	소설	NNG	O
의	의	JKG	O
_	_	_	O
삽화	삽화	NNG	O
로	로	JKB	O
_	_	_	O
동원하	동원하	VV	O
며	며	EC	O
_	_	_	O
일본	일본	NNP	B-ORG
_	_	_	O
사회	사회	NNG	O
의	의	JKG	O
_	_	_	O
‘	‘	SS	O
비정상	비정상	NNG	O
성	성	XSN	O
’	’	SS	O
을	을	JKO	O
_	_	_	O
문제	문제	NNG	O
_	_	_	O
삼	삼	VV	O
는다	는다	EF	O
.	.	SF	O"""

In [150]:
texts = [ line1.split('\t')[0] for line1 in test.split('\n') if line1.split('\t')[2] != '_' ]
#print(texts)

In [2]:
# SKT KoBERT
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import BertTokenizer, BertModel, BertConfig

In [3]:
# KoBERT tokenizer
kobert, vocab = get_pytorch_kobert_model()
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

using cached model
using cached model
using cached model


In [153]:
text = "오에 겐자부로는 일본 현대문학의 초석을 놓은 것으로 평가받는 작가 나쓰메 소세키(1867~1916)의 대표작 ‘마음’에 담긴 군국주의적 요소, 야스쿠니 신사 참배 행위까지 소설의 삽화로 동원하며 일본 사회의 ‘비정상성’을 문제 삼는다."
tag_text = "<오에 겐자부로:PER>는 <일본:LOC> 현대문학의 초석을 놓은 것으로 평가받는 작가 <나쓰메 소세키:PER>(<1867~1916:DUR>)의 대표작 ‘<마음:POH>’에 담긴 군국주의적 요소, <야스쿠니 신사:ORG> 참배 행위까지 소설의 삽화로 동원하며 <일본:ORG> 사회의 ‘비정상성’을 문제 삼는다."
tokens = tokenizer(text)
print(tokens)

['▁오', '에', '▁', '겐', '자', '부', '로', '는', '▁일본', '▁현대', '문학', '의', '▁초', '석', '을', '▁놓', '은', '▁것으로', '▁평가', '받는', '▁작가', '▁나', '쓰', '메', '▁소', '세', '키', '▁(', '▁18', '67', '▁', '~', '▁19', '16', '▁', ')', '▁', '의', '▁대표', '작', '▁‘', '▁마음', '▁', '’', '▁', '에', '▁담긴', '▁군', '국', '주의', '적', '▁요소', '▁', ',', '▁야', '스', '쿠', '니', '▁신', '사', '▁참', '배', '▁행위', '까지', '▁소설', '의', '▁', '삽', '화', '로', '▁동원', '하며', '▁일본', '▁사회', '의', '▁‘', '▁비', '정', '상', '성', '▁', '’', '▁', '을', '▁문제', '▁삼', '는', '다', '▁', '.']


In [154]:
ret = []
for text1 in texts:
    ret.extend(tokenizer(text1))
print(texts)
print()
print(ret)
ret == tokens

['오에', '겐자부로', '는', '일본', '현대', '문학', '의', '초석', '을', '놓', '은', '것', '으로', '평가', '받', '는', '작가', '나쓰메', '소세키', '(', '1867', '~', '1916', ')', '의', '대표작', '‘', '마음', '’', '에', '담긴', '군국주의', '적', '요소', ',', '야스쿠니', '신사', '참배', '행위', '까지', '소설', '의', '삽화', '로', '동원하', '며', '일본', '사회', '의', '‘', '비정상', '성', '’', '을', '문제', '삼', '는다', '.']

['▁오', '에', '▁', '겐', '자', '부', '로', '▁', '는', '▁일본', '▁현대', '▁', '문학', '▁', '의', '▁초', '석', '▁', '을', '▁놓', '▁', '은', '▁것', '▁', '으로', '▁평가', '▁받', '▁', '는', '▁작가', '▁나', '쓰', '메', '▁소', '세', '키', '▁(', '▁18', '67', '▁', '~', '▁19', '16', '▁', ')', '▁', '의', '▁대표', '작', '▁‘', '▁마음', '▁', '’', '▁', '에', '▁담긴', '▁군', '국', '주의', '▁적', '▁요소', '▁', ',', '▁야', '스', '쿠', '니', '▁신', '사', '▁참', '배', '▁행위', '▁', '까지', '▁소설', '▁', '의', '▁', '삽', '화', '▁', '로', '▁동원', '하', '▁', '며', '▁일본', '▁사회', '▁', '의', '▁‘', '▁비', '정', '상', '▁성', '▁', '’', '▁', '을', '▁문제', '▁삼', '▁', '는', '다', '▁', '.']


False

In [155]:
print(tokenizer.vocab.idx_to_token[3])
print(tokenizer.vocab.idx_to_token[25])
print(tokenizer.vocab.idx_to_token[199])
print(tokenizer.vocab.idx_to_token[522])
print(tokenizer.vocab.idx_to_token[1])

[SEP]
(18
67
▁(
[PAD]


In [156]:
print(tokenizer.vocab.token_to_idx['▁'])
print(tokenizer.vocab.token_to_idx['▁'])
print(tokenizer.vocab.token_to_idx['_'])

517
517
365


In [157]:
arr = '3417 6896  517 5402 7147 6398 6079 5760 3809 5051 6237 7095 4501'.split()
for x in arr:
    print(tokenizer.vocab.idx_to_token[int(x)])

▁오
에
▁
겐
자
부
로
는
▁일본
▁현대
문학
의
▁초


In [4]:
# BERT tokenizer
B_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [170]:
type(B_tokenizer)

transformers.tokenization_bert.BertTokenizer

In [5]:
def split_tokenizer(text):
    return text.split()

def token_loc(text, tokenizer):
    """
    For a given tokenizer, obtain each token and its position in the original text
    """
    if isinstance(tokenizer, BertTokenizer):
        tokens = tokenizer.tokenize(text)
    else:
        tokens = tokenizer(text)
    token_word_loc = []
    i_pos = 0
    for token in tokens:
        tokenp = token
        if '▁' in token: # padding for word2piece
            if len(token) > 1:
                tokenp = token[1:]
            else:
                tokenp = ' '
        i_pos, f_pos = find_loc(text, tokenp, start_i=i_pos)
        temp = (token, (i_pos, f_pos))
        token_word_loc.append(temp)
        i_pos = f_pos
    return token_word_loc
# test
split_tokens = token_loc(text, split_tokenizer)
print(split_tokens)
print()
wp_tokens = token_loc(text, tokenizer)
print(wp_tokens)
print()
B_tokens = token_loc(text, B_tokenizer)
print(B_tokens)

[('카카오뱅크는', (0, 6)), ('오는', (7, 9)), ('13일부터', (10, 15)), ('다음', (16, 18)), ('달', (19, 20)), ('2일까지', (21, 25)), ("'가져오기'", (26, 32)), ('기능을', (33, 36)), ('실행하고', (37, 41)), ('이벤트에', (42, 46)), ('참여한', (47, 50)), ('고객에게', (51, 55)), ('아이패드,', (56, 61)), ('애플워치,', (62, 67)), ('편의점', (68, 71)), ('상품권', (72, 75)), ('등을', (76, 78)), ('제공하는', (79, 83)), ('이벤트도', (84, 88)), ('진행한다.', (89, 94))]

[('▁카카오', (0, 3)), ('뱅', (3, 4)), ('크', (4, 5)), ('는', (5, 6)), ('▁오는', (7, 9)), ('▁13', (10, 12)), ('일부터', (12, 15)), ('▁다음', (16, 18)), ('▁달', (19, 20)), ('▁2', (21, 22)), ('일까지', (22, 25)), ("▁'", (26, 27)), ('▁가져', (27, 29)), ('오', (29, 30)), ('기', (30, 31)), ("▁'", (31, 32)), ('▁기능을', (33, 36)), ('▁실행', (37, 39)), ('하고', (39, 41)), ('▁이벤트', (42, 45)), ('에', (45, 46)), ('▁참여한', (47, 50)), ('▁고객에게', (51, 55)), ('▁아이', (56, 58)), ('패', (58, 59)), ('드', (59, 60)), ('▁', (60, 60)), (',', (60, 61)), ('▁애플', (62, 64)), ('워', (64, 65)), ('치', (65, 66)), ('▁', (66, 66)), (',', (66, 67)), ('▁편의점', (68, 71

In [6]:
def find_loc(text, subword, start_i=0):
    """
    In a text, gives index of subword from start_i
    in case subword is space, prohibit it matches with spaces 
    behind of sentences
    :return: 
    i_loc, f_loc
    """
    if subword == ' ':
        if text[start_i] == ' ':
            return start_i, start_i+1
        else:
            return start_i, start_i
    try:
        i_loc = start_i + text[start_i:].index(subword)
        f_loc = i_loc + len(subword)
    except:
        i_loc = start_i
        f_loc = i_loc
    
    return i_loc, f_loc
# test
#find_loc('I love youy love', 'love', start_i = 5)

import re
def get_NE(tag_text, text, ne_exp='<(.+?):([A-Z]{3})>'):
    """
    find Named Entity from given text
    :return: 
    [[NE, TAG, (i_pos, f_pos)]]
    """
    ne_word_loc = []
    reg_exp = re.compile(ne_exp)
    nes = reg_exp.finditer(tag_text)
    i_pos = 0
    for ne in nes:
        subword, tag = ne[1], ne[2]
        i_pos, f_pos = find_loc(text, subword, start_i=i_pos)
        temp = [subword, tag, (i_pos, f_pos)]
        ne_word_loc.append(temp)
    return ne_word_loc
    
ne_exp = '<(.+?):([A-Z]{3})>'

text = "카카오뱅크는 오는 13일부터 다음 달 2일까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 아이패드, 애플워치, 편의점 상품권 등을 제공하는 이벤트도 진행한다.\n"
tag_text = "<카카오뱅크:ORG>는 오는 <13일:DAT>부터 다음 달 <2일:DAT>까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 <아이패드:POH>, <애플워치:POH>, <편의점 상품권:POH> 등을 제공하는 이벤트도 진행한다.\n"

print('tag_text', tag_text)
print('text', text)
print('ne_+exp', ne_exp)
ner_tokens = get_NE(tag_text, text, ne_exp)
print(ner_ids)
print(ner_tokens)

tag_text <카카오뱅크:ORG>는 오는 <13일:DAT>부터 다음 달 <2일:DAT>까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 <아이패드:POH>, <애플워치:POH>, <편의점:POH>< 상품권:POH> 등을 제공하는 이벤트도 진행한다.

text 카카오뱅크는 오는 13일부터 다음 달 2일까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 아이패드, 애플워치, 편의점 상품권 등을 제공하는 이벤트도 진행한다.

ne_+exp <(.+?):([A-Z]{3})>


NameError: name 'ner_ids' is not defined

In [7]:
def match(tag_tokens, model_tokens):
    """
    :in: 
    tokens1, tokens2 are in form of [(word, pos)], pos = (i_loc, f_loc)
    f_loc is inclusive
    :desc:
    if some forms are not adequate like empty subword, it can be removed by post-processing
    """
    matching_ids = []
    for tgt_token in tag_tokens:
        tgt_i_loc, tgt_f_loc = tgt_token[-1]
        i_id, f_id = 0, 0
        for i_token, token in enumerate(model_tokens):
            i_loc, f_loc = token[-1]
            # case1
            if tgt_i_loc >= i_loc and tgt_i_loc < f_loc:
                i_id = i_token
            # case2
            if tgt_f_loc > i_loc and tgt_i_loc <= f_loc:
                f_id = i_token
        matching_ids.append((i_id, f_id))
    return matching_ids

## Main (preprocessing)
get tags from matching ids of (tagging ner_tokens, model wp_tokens)

In [8]:
ner_tokens

[['카카오뱅크', 'ORG', (0, 5)],
 ['13일', 'DAT', (10, 13)],
 ['2일', 'DAT', (21, 23)],
 ['아이패드', 'POH', (56, 60)],
 ['애플워치', 'POH', (62, 66)],
 ['편의점', 'POH', (68, 71)],
 [' 상품권', 'POH', (71, 75)]]

In [9]:
wp_tokens

[('▁카카오', (0, 3)),
 ('뱅', (3, 4)),
 ('크', (4, 5)),
 ('는', (5, 6)),
 ('▁오는', (7, 9)),
 ('▁13', (10, 12)),
 ('일부터', (12, 15)),
 ('▁다음', (16, 18)),
 ('▁달', (19, 20)),
 ('▁2', (21, 22)),
 ('일까지', (22, 25)),
 ("▁'", (26, 27)),
 ('▁가져', (27, 29)),
 ('오', (29, 30)),
 ('기', (30, 31)),
 ("▁'", (31, 32)),
 ('▁기능을', (33, 36)),
 ('▁실행', (37, 39)),
 ('하고', (39, 41)),
 ('▁이벤트', (42, 45)),
 ('에', (45, 46)),
 ('▁참여한', (47, 50)),
 ('▁고객에게', (51, 55)),
 ('▁아이', (56, 58)),
 ('패', (58, 59)),
 ('드', (59, 60)),
 ('▁', (60, 60)),
 (',', (60, 61)),
 ('▁애플', (62, 64)),
 ('워', (64, 65)),
 ('치', (65, 66)),
 ('▁', (66, 66)),
 (',', (66, 67)),
 ('▁편의점', (68, 71)),
 ('▁상품', (72, 74)),
 ('권', (74, 75)),
 ('▁등을', (76, 78)),
 ('▁제공하는', (79, 83)),
 ('▁이벤트', (84, 87)),
 ('도', (87, 88)),
 ('▁진행한다', (89, 93)),
 ('▁', (93, 93)),
 ('.', (93, 94))]

In [10]:
# get matching ids
matching_ids = match(ner_tokens, wp_tokens)

# preprocess (TAG --> B-TAG or I-TAG)
tg_tags = [(i, x[1]) for i, x in enumerate(ner_tokens)]
tags = [None]*len(wp_tokens)
for ii, tag in tg_tags:
    i_id, f_id = matching_ids[ii]
    tags[i_id] = 'B-'+tag
    for jj in range(i_id+1, f_id+1):
        tags[jj] = 'I-'+tag

# test
for ii, tag in enumerate(wp_tokens):
    print(tags[ii], tag[0])
    
""" test
matching_ids = match(split_tokens, wp_tokens)
ii = 0
for i_id, f_id in matching_ids:
    #print(i_id, f_id)
    print(split_tokens[ii][0], wp_tokens[i_id][0], wp_tokens[f_id][0])
    ii += 1
"""    

B-POH ▁카카오
I-POH 뱅
I-POH 크
I-POH 는
I-POH ▁오는
I-POH ▁13
I-POH 일부터
I-POH ▁다음
I-POH ▁달
I-POH ▁2
I-POH 일까지
I-POH ▁'
I-POH ▁가져
I-POH 오
I-POH 기
I-POH ▁'
I-POH ▁기능을
I-POH ▁실행
I-POH 하고
I-POH ▁이벤트
I-POH 에
I-POH ▁참여한
I-POH ▁고객에게
I-POH ▁아이
I-POH 패
I-POH 드
I-POH ▁
I-POH ,
I-POH ▁애플
I-POH 워
I-POH 치
I-POH ▁
I-POH ,
I-POH ▁편의점
I-POH ▁상품
I-POH 권
None ▁등을
None ▁제공하는
None ▁이벤트
None 도
None ▁진행한다
None ▁
None .


' test\nmatching_ids = match(split_tokens, wp_tokens)\nii = 0\nfor i_id, f_id in matching_ids:\n    #print(i_id, f_id)\n    print(split_tokens[ii][0], wp_tokens[i_id][0], wp_tokens[f_id][0])\n    ii += 1\n'

In [144]:
tg_tags

[(0, 'PER'),
 (1, 'LOC'),
 (2, 'PER'),
 (3, 'DUR'),
 (4, 'POH'),
 (5, 'ORG'),
 (6, 'ORG')]

In [79]:
for i, x in enumerate(text):
    print(i, x, end=' ')

0 오 1 에 2   3 겐 4 자 5 부 6 로 7 는 8   9 일 10 본 11   12 현 13 대 14 문 15 학 16 의 17   18 초 19 석 20 을 21   22 놓 23 은 24   25 것 26 으 27 로 28   29 평 30 가 31 받 32 는 33   34 작 35 가 36   37 나 38 쓰 39 메 40   41 소 42 세 43 키 44 ( 45 1 46 8 47 6 48 7 49 ~ 50 1 51 9 52 1 53 6 54 ) 55 의 56   57 대 58 표 59 작 60   61 ‘ 62 마 63 음 64 ’ 65 에 66   67 담 68 긴 69   70 군 71 국 72 주 73 의 74 적 75   76 요 77 소 78 , 79   80 야 81 스 82 쿠 83 니 84   85 신 86 사 87   88 참 89 배 90   91 행 92 위 93 까 94 지 95   96 소 97 설 98 의 99   100 삽 101 화 102 로 103   104 동 105 원 106 하 107 며 108   109 일 110 본 111   112 사 113 회 114 의 115   116 ‘ 117 비 118 정 119 상 120 성 121 ’ 122 을 123   124 문 125 제 126   127 삼 128 는 129 다 130 . 

In [124]:
wp_tokens[2][0]

'▁'

In [125]:
split_tokens

[('오에', (0, 2)),
 ('겐자부로는', (3, 8)),
 ('일본', (9, 11)),
 ('현대문학의', (12, 17)),
 ('초석을', (18, 21)),
 ('놓은', (22, 24)),
 ('것으로', (25, 28)),
 ('평가받는', (29, 33)),
 ('작가', (34, 36)),
 ('나쓰메', (37, 40)),
 ('소세키(1867~1916)의', (41, 56)),
 ('대표작', (57, 60)),
 ('‘마음’에', (61, 66)),
 ('담긴', (67, 69)),
 ('군국주의적', (70, 75)),
 ('요소,', (76, 79)),
 ('야스쿠니', (80, 84)),
 ('신사', (85, 87)),
 ('참배', (88, 90)),
 ('행위까지', (91, 95)),
 ('소설의', (96, 99)),
 ('삽화로', (100, 103)),
 ('동원하며', (104, 108)),
 ('일본', (109, 111)),
 ('사회의', (112, 115)),
 ('‘비정상성’을', (116, 123)),
 ('문제', (124, 126)),
 ('삼는다.', (127, 131))]